"""
Created by Yachay Tolosa - yachay@gmail.com
"""

modificado por Ivan Corredor ivangcorredorc@gmail.com


In [350]:
#!pip install --upgrade pandas=='0.24.2'

In [351]:
#pd.__version__

In [352]:
import pandas as pd
import numpy as np
from datetime import datetime

In [353]:
#rutas: 
p1 = "C:/Users/ivan.corredor/IGED_justicia/"
p2 = "C:/Users/ivan.corredor/IGED_justicia/IGED FISCALIA/"

In [354]:
### cargue base unidad para las víctimas
data = pd.read_csv(p1 + "hechos_mpio_dia.csv",sep= ",", parse_dates=[0])


In [355]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133259 entries, 0 to 1133258
Data columns (total 3 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   F_OCURRENCIA_HECHO  1133259 non-null  datetime64[ns]
 1   DANE_SINIESTRO      1133244 non-null  float64       
 2   CONSPERSONA         1133259 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 25.9 MB


In [356]:
#colocar nombres de columns en minuscula
data.columns = data.columns.str.lower()

In [357]:
data.columns

Index(['f_ocurrencia_hecho', 'dane_siniestro', 'conspersona'], dtype='object')

In [358]:
## reemplazar nombres de variables:
data.rename(columns={"f_ocurrencia_hecho":"fechaocurrencia","dane_siniestro":"codmpio"}, inplace=True)

In [359]:
data.sort_values('fechaocurrencia', ascending=False)['fechaocurrencia'].head(5)

871   2072-11-15
870   2072-10-28
869   2072-10-10
849   2072-10-03
865   2072-07-23
Name: fechaocurrencia, dtype: datetime64[ns]

In [360]:
#fintra las ocurrencias entre 1985 y 31 de diciembre de 2021
data_ff = data[(data.fechaocurrencia >= datetime(1985, 1, 1)) & (data.fechaocurrencia <= datetime(2022, 12, 31))] #datetime.today())]

In [361]:
data_ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1131537 entries, 0 to 1133258
Data columns (total 3 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   fechaocurrencia  1131537 non-null  datetime64[ns]
 1   codmpio          1131522 non-null  float64       
 2   conspersona      1131537 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 34.5 MB


In [362]:
data_ff.sort_values('fechaocurrencia', ascending=False)['fechaocurrencia'].head(5)

1133258   2022-12-31
1133246   2022-12-31
1133235   2022-12-31
1133236   2022-12-31
1133237   2022-12-31
Name: fechaocurrencia, dtype: datetime64[ns]

In [363]:
#valdiar fecha máxima y minima
print(data_ff["fechaocurrencia"].min(), data_ff["fechaocurrencia"].max())


1985-01-01 00:00:00 2022-12-31 00:00:00


In [364]:
# Crea una columna con el número de días transcurridos desde 01/01/1970
data_ff["dia"] = (data_ff.fechaocurrencia - datetime(1970, 1, 1)).dt.days

C:\Users\ivan.corredor\AppData\Local\Temp\3\ipykernel_12192\2504430387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ff["dia"] = (data_ff.fechaocurrencia - datetime(1970, 1, 1)).dt.days


In [365]:
# Guarda la información
# This format is useful for preserving the data types and other metadata associated with the DataFrame.
# to preserve the data types and metadata associated with a pandas DataFrame, or if you plan to use the DataFrame in another Python application
data_ff.to_pickle(p1 + "hechos_mpio_dia.pkl")

### histórico fiscalía 2020 más periodos 2021 y 2022
Se cargan los datos de fiscalía a 2020 de la pestañana SIJYP, los cuales son históricos. Para 2021, 2022 y 2023, la fiscalía solo entrega los nuevos datos de SIJYP en cada año, por lo que se debe pegar esta base y armar el histórico a 2023.

In [366]:
### cargar hisórico, años y estandarizar
#2020

SIJYP_20 = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_histórico_SIJYP_31_12_2019_2020.xlsx", sheet_name="SIJYP",
                         parse_dates=[2,3])
#2021

SIJYP_21 = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_2021.xlsx", sheet_name="SIJYP",
                         parse_dates=[5,6])
#2022

SIJYP_22 = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_2022.xlsx", sheet_name="SIJYP",
                         parse_dates=[4,5])

SIJYP_23 = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_2023.xlsx", sheet_name="SIJYP",parse_dates=[4,5])

In [367]:
### cargue de las dos pestañas de la base de la fiscalia y concatenación
# 2020
SPOA_SIJUF_20 = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_histórico_SIJYP_31_12_2019_2020.xlsx",  sheet_name="SPOA_SIJUF",parse_dates=[3,4])
# 2021-2023
for i in range(21,24):
    exec('SPOA_SIJUF_{} = pd.read_excel(p2 + "IGED_DESPLAZAMIENTO_INDICADOR1_20{}.xlsx",  sheet_name="SPOA_SIJUF",parse_dates=[3,4])'.format(i,i))

In [368]:
# id_caso, codmpio, fecha_hechos_inicial, fecha_hechos_final, base_origen
#cambiar nombres de variables base SPOA_SIJUF

#renombrar
for i in range(20,24):
    columns_names1 = {"CASO_ID": "id_caso", "CODIGO_DANE_MUN_HECHOS":"codmpio",
                           "FECHA_INICIO_HECHOS":"fecha_hechos_inicial", "FECHA_FINAL_HECHOS":"fecha_hechos_final", 
                           "SISTEMA":"base_origen"}
    exec('SPOA_SIJUF_{}.rename(columns=columns_names1, inplace= True)'.format(i))
    #seleccionar variables requeridas
    exec('SPOA_SIJUF_{} = SPOA_SIJUF_{}[["id_caso", "codmpio","fecha_hechos_inicial", "fecha_hechos_final", "base_origen"]]'.format(i,i))

In [369]:
#valdiar fecha máxima y minima
print(SPOA_SIJUF_23["fecha_hechos_final"].min(), SPOA_SIJUF_23["fecha_hechos_final"].max())
print(SPOA_SIJUF_23["fecha_hechos_inicial"].min(), SPOA_SIJUF_23["fecha_hechos_inicial"].max())

1977-03-10 00:00:00 2022-12-28 11:53:00
1957-04-15 23:30:00 2022-12-29 08:00:00


In [370]:
#cambiar nombres de variables base SIJYP histórico 2020-2023
#renombrar
for i in range(20,24):
    columns_names = {"IDCARPETAHECHO": "id_caso", "Idmpio":"codmpio", "FECHAINICIALEVENTO":"fecha_hechos_inicial", "FECHAFINALEVENTO":"fecha_hechos_final","SISTEMA":"base_origen"}
    exec('SIJYP_{}.rename(columns=columns_names, inplace= True)'.format(i))
    #seleccionar variables requeridas
    exec('SIJYP_{} = SIJYP_{}[["id_caso", "codmpio","fecha_hechos_inicial", "fecha_hechos_final", "base_origen"]]'.format(i,i))

In [371]:
print(SIJYP_21["fecha_hechos_final"].min(), SIJYP_21["fecha_hechos_final"].max())

1948-04-10 00:00:00 2021-01-26 00:00:00


In [372]:
### concatenar y cambiar formato de variables a fecha

justicia = pd.concat([SPOA_SIJUF_20, SPOA_SIJUF_21,SPOA_SIJUF_22,SPOA_SIJUF_23, SIJYP_20, SIJYP_21, SIJYP_22, SIJYP_23], axis=0)

In [374]:
justicia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1035439 entries, 0 to 8388
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   id_caso               1035439 non-null  object        
 1   codmpio               1035361 non-null  object        
 2   fecha_hechos_inicial  999569 non-null   datetime64[ns]
 3   fecha_hechos_final    774430 non-null   datetime64[ns]
 4   base_origen           1035439 non-null  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 47.4+ MB


In [375]:
# Crea una columna con el número de días transcurridos desde 01/01/1970
justicia["final"] = (pd.to_datetime(justicia.fecha_hechos_final, format="%d%b%Y") - datetime(1970, 1, 1)).dt.days
justicia["inicial"] = (pd.to_datetime(justicia.fecha_hechos_inicial, format="%d%b%Y") - datetime(1970, 1, 1)).dt.days

In [376]:
# Elimina los registros en que la fecha final es anterior a la inicial
justicia = justicia[justicia.inicial <= justicia.final]

In [377]:
# Fija los intervalos mayores a 365 días a 7 días
justicia.loc[justicia.final - justicia.inicial > 365, "final"] = justicia.loc[
                                                                     justicia.final - justicia.inicial > 365, "inicial"] + 7

In [378]:
# Elimina Duplicados
justicia = justicia.drop_duplicates()

In [379]:
justicia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334346 entries, 0 to 8354
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id_caso               334346 non-null  object        
 1   codmpio               334291 non-null  object        
 2   fecha_hechos_inicial  334346 non-null  datetime64[ns]
 3   fecha_hechos_final    334346 non-null  datetime64[ns]
 4   base_origen           334346 non-null  object        
 5   final                 334346 non-null  float64       
 6   inicial               334346 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 20.4+ MB


In [380]:
justicia.head()

,id_caso,codmpio,fecha_hechos_inicial,fecha_hechos_final,base_origen,final,inicial
0,111085_11,5031.0,2003-09-09,2003-09-09,SIJUF,12304.0,12304.0
1,111021_11,5234.0,2003-10-28,2003-10-28,SIJUF,12353.0,12353.0
2,169470_11,5847.0,2004-10-01,2004-10-01,SIJUF,12692.0,12692.0
3,170117_11,5040.0,2002-10-01,2002-10-01,SIJUF,11961.0,11961.0
4,170118_11,5652.0,2004-01-01,2004-01-01,SIJUF,12418.0,12418.0


In [381]:
# Guarda la información
justicia.to_pickle(p1 + "justicia.pkl")

In [382]:
#!pip install --upgrade scikit-learn

In [383]:
#sklearn.__version__

In [384]:
#importar librerias
from scipy.signal import argrelextrema
import sklearn
from sklearn.neighbors import KernelDensity

In [385]:
#definir kernel y cargar divipola
kernel = 'gaussian'

municipios = pd.read_csv(p1 + "DIVIPOLA_20170330.csv", sep=';').CodMun

In [386]:
print(data_ff.dia.max(), data_ff.conspersona.max())

19357 11812


In [387]:
##obtener el municipio con más ocurrencias y más personas
ff =  data_ff.reset_index(drop=True).groupby(["codmpio"]).aggregate({"codmpio": np.size, "conspersona":np.sum})
ff.index.names = ["municipio"]
ff = ff.sort_values(by=["codmpio", "conspersona"], ascending=[False, False]).reset_index()
ff[(ff["codmpio"]>1000) & (ff["conspersona"]>50000)].head(20)


,municipio,codmpio,conspersona
0,5837.0,9447,152582
1,5001.0,9197,146715
2,23807.0,8208,132770
3,5045.0,8055,84381
4,76109.0,7915,379096
5,52835.0,7624,210149
7,18753.0,6155,57070
8,54810.0,6092,84546
9,47001.0,6010,103116
10,27001.0,5971,58936


In [388]:
#cuenta la cantidad de municipios mayor a 1000 ocurrencias
ff[ff["codmpio"]>1000].shape[0]

#ff[ff["codmpio"]<1000]["codmpio"].size

360

In [389]:
municipios[municipios.to_frame().CodMun == 15762]

294    15762
Name: CodMun, dtype: int64

In [390]:
municipios.head()

0    5001
1    5002
2    5004
3    5021
4    5030
Name: CodMun, dtype: int64

In [391]:
# Variable que contiene el 01-01-1985 en días desde 1970
inicio = (datetime(1985, 1, 1) - datetime(1970, 1, 1)).days

KDE_result = None
# Hace el cálculo para cada municipio
avance = 0
for mun in municipios:
    avance += 1
    print("Procesando: {} de {}".format(avance, len(municipios)))
    dataMun = data_ff[data_ff.codmpio == mun].copy()

    # tiene en cuenta sólo los municipios que tienen al menos 2 datos
    if dataMun["conspersona"].count() < 2:
        continue

    # se transforman los datos para tener en cuenta el peso
    A = dataMun.dia
    B = dataMun.conspersona  # Esta variable contiene el número de personas que representa cada registro
    #example: the first pair of values from zip(A, B) is (10592, 2), so 10592 is repeated twice. The fifth pair of values is (5613, 5), so 5613 is repeated five times. And so on.
    #this list is converted to a numpy array and reshaped into a single column matrix with as many rows as there are elements in the list
    C = np.array([a for a, b in zip(A, B) for i in range(b)]).reshape(-1, 1)

    # Se calcula el bandwith siguiendo la regla de Silverman
    #A smaller bandwidth will result in a more "peaky" estimate, while a larger bandwidth will result in a smoother estimate.
    bw = (C.size * 3 / 4.) ** (-1. / 5)

    # Se estima el KDE
    kde = KernelDensity(kernel=kernel, bandwidth=bw).fit(C)
    
    #example: if the minimum value in C is 5479 and the maximum value is 18067, then np.linspace(C.min(), C.max(), C.max() - C.min()) 
    #will generate an array with 18067 values, evenly spaced between 5479 and 18067 (excluding the endpoints)
    #number of points to generate (C.max() - C.min()).
    s = np.linspace(C.min(), C.max(), C.max() - C.min())
    
    #obtain the log-density values for each point in the sequence (day's numbers since 1970 to incidence date).
    #The resulting log-density values are stored in the e variable.
    e = kde.score_samples(s.reshape(-1, 1))

    # Agrega el inicio para no tener en cuanta la distribucion antes de 1985
    #defining the bin edges for the histogram of a data set C by finding the local minima of the estimated density function e 
    #and appending them to the minimum and maximum values of C.
    #p.less: selects the indices where the array is less than its neighbors.
    mi = np.append([inicio], argrelextrema(e, np.less)[0] + min(C))
    mi = np.append(mi, [C.max()])

    # Cuenta el número de eventos
    
    dataMun = dataMun.reset_index(drop=True)
    #pd.cut function takes three arguments: the column to be binned (dataMun.dia), the bin edges (mi), and the labels to use for the resulting bins. 
    #The mi variable contains the bin edges, with mi[:-1] being used as the labels for the left edges of the bins 
    #and mi[1:] being used as the labels for the right edges of the bins.
    
    F = dataMun.groupby(
        [pd.cut(dataMun.dia, mi, labels=mi[:-1]), pd.cut(dataMun.dia, mi, labels=mi[1:]), "codmpio"])[
        "conspersona"].sum().to_frame()
    G = F[F.conspersona >=5]
    #borra cache para mejorar consumo de RAM
    del F, A, B, C, bw, kde, s, e, mi
    # Estandariza los nombres y tipos de la tabla
    #rename index names
    G.index.names = ['inicial', 'final', 'codmpio']
    #reset index value
    G = G.reset_index()
    #convert to number
    G.inicial = pd.to_numeric(G.inicial)
    G.final = pd.to_numeric(G.final)

    # Ecribe la información en el archivo
    if KDE_result is None:
        KDE_result = G
    else:
        KDE_result = pd.concat([KDE_result, G])
        
#24 minutos 30 segundos de ejecución

Procesando: 1 de 1122
Procesando: 2 de 1122
Procesando: 3 de 1122
Procesando: 4 de 1122
Procesando: 5 de 1122
Procesando: 6 de 1122
Procesando: 7 de 1122
Procesando: 8 de 1122
Procesando: 9 de 1122
Procesando: 10 de 1122
Procesando: 11 de 1122
Procesando: 12 de 1122
Procesando: 13 de 1122
Procesando: 14 de 1122
Procesando: 15 de 1122
Procesando: 16 de 1122
Procesando: 17 de 1122
Procesando: 18 de 1122
Procesando: 19 de 1122
Procesando: 20 de 1122
Procesando: 21 de 1122
Procesando: 22 de 1122
Procesando: 23 de 1122
Procesando: 24 de 1122
Procesando: 25 de 1122
Procesando: 26 de 1122
Procesando: 27 de 1122
Procesando: 28 de 1122
Procesando: 29 de 1122
Procesando: 30 de 1122
Procesando: 31 de 1122
Procesando: 32 de 1122
Procesando: 33 de 1122
Procesando: 34 de 1122
Procesando: 35 de 1122
Procesando: 36 de 1122
Procesando: 37 de 1122
Procesando: 38 de 1122
Procesando: 39 de 1122
Procesando: 40 de 1122
Procesando: 41 de 1122
Procesando: 42 de 1122
Procesando: 43 de 1122
Procesando: 44 de 11

Procesando: 347 de 1122
Procesando: 348 de 1122
Procesando: 349 de 1122
Procesando: 350 de 1122
Procesando: 351 de 1122
Procesando: 352 de 1122
Procesando: 353 de 1122
Procesando: 354 de 1122
Procesando: 355 de 1122
Procesando: 356 de 1122
Procesando: 357 de 1122
Procesando: 358 de 1122
Procesando: 359 de 1122
Procesando: 360 de 1122
Procesando: 361 de 1122
Procesando: 362 de 1122
Procesando: 363 de 1122
Procesando: 364 de 1122
Procesando: 365 de 1122
Procesando: 366 de 1122
Procesando: 367 de 1122
Procesando: 368 de 1122
Procesando: 369 de 1122
Procesando: 370 de 1122
Procesando: 371 de 1122
Procesando: 372 de 1122
Procesando: 373 de 1122
Procesando: 374 de 1122
Procesando: 375 de 1122
Procesando: 376 de 1122
Procesando: 377 de 1122
Procesando: 378 de 1122
Procesando: 379 de 1122
Procesando: 380 de 1122
Procesando: 381 de 1122
Procesando: 382 de 1122
Procesando: 383 de 1122
Procesando: 384 de 1122
Procesando: 385 de 1122
Procesando: 386 de 1122
Procesando: 387 de 1122
Procesando: 388 

Procesando: 689 de 1122
Procesando: 690 de 1122
Procesando: 691 de 1122
Procesando: 692 de 1122
Procesando: 693 de 1122
Procesando: 694 de 1122
Procesando: 695 de 1122
Procesando: 696 de 1122
Procesando: 697 de 1122
Procesando: 698 de 1122
Procesando: 699 de 1122
Procesando: 700 de 1122
Procesando: 701 de 1122
Procesando: 702 de 1122
Procesando: 703 de 1122
Procesando: 704 de 1122
Procesando: 705 de 1122
Procesando: 706 de 1122
Procesando: 707 de 1122
Procesando: 708 de 1122
Procesando: 709 de 1122
Procesando: 710 de 1122
Procesando: 711 de 1122
Procesando: 712 de 1122
Procesando: 713 de 1122
Procesando: 714 de 1122
Procesando: 715 de 1122
Procesando: 716 de 1122
Procesando: 717 de 1122
Procesando: 718 de 1122
Procesando: 719 de 1122
Procesando: 720 de 1122
Procesando: 721 de 1122
Procesando: 722 de 1122
Procesando: 723 de 1122
Procesando: 724 de 1122
Procesando: 725 de 1122
Procesando: 726 de 1122
Procesando: 727 de 1122
Procesando: 728 de 1122
Procesando: 729 de 1122
Procesando: 730 

Procesando: 1030 de 1122
Procesando: 1031 de 1122
Procesando: 1032 de 1122
Procesando: 1033 de 1122
Procesando: 1034 de 1122
Procesando: 1035 de 1122
Procesando: 1036 de 1122
Procesando: 1037 de 1122
Procesando: 1038 de 1122
Procesando: 1039 de 1122
Procesando: 1040 de 1122
Procesando: 1041 de 1122
Procesando: 1042 de 1122
Procesando: 1043 de 1122
Procesando: 1044 de 1122
Procesando: 1045 de 1122
Procesando: 1046 de 1122
Procesando: 1047 de 1122
Procesando: 1048 de 1122
Procesando: 1049 de 1122
Procesando: 1050 de 1122
Procesando: 1051 de 1122
Procesando: 1052 de 1122
Procesando: 1053 de 1122
Procesando: 1054 de 1122
Procesando: 1055 de 1122
Procesando: 1056 de 1122
Procesando: 1057 de 1122
Procesando: 1058 de 1122
Procesando: 1059 de 1122
Procesando: 1060 de 1122
Procesando: 1061 de 1122
Procesando: 1062 de 1122
Procesando: 1063 de 1122
Procesando: 1064 de 1122
Procesando: 1065 de 1122
Procesando: 1066 de 1122
Procesando: 1067 de 1122
Procesando: 1068 de 1122
Procesando: 1069 de 1122


In [392]:
#exportar KDE_result
KDE_result.to_pickle(p1 + 'KDE_result.pkl')

In [393]:
KDE_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457654 entries, 0 to 1130
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   inicial      457654 non-null  int64  
 1   final        457654 non-null  int64  
 2   codmpio      457654 non-null  float64
 3   conspersona  457654 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 17.5 MB


### importe de bases procesadas

In [394]:
#import pandas as pd
#import numpy as np
import sqlite3

In [395]:
### importar bases para no volver a correr todo:
#KDE_result = pd.read_pickle('C:/Users/ivan.corredor/IGED_justicia/validación_2022/KDE_result.pkl')
#justicia = pd.read_pickle("C:/Users/ivan.corredor/IGED_justicia/validación_2022/justicia.pkl")

In [396]:
#agregar cero a la izquierda de variable cdmpio
KDE_result["codmpio"] = KDE_result["codmpio"].astype(int)
KDE_result["codmpio"] = KDE_result["codmpio"].astype(str).str.zfill(5) 

In [397]:
justicia = justicia.reset_index(drop = True)

In [398]:
justicia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334346 entries, 0 to 334345
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id_caso               334346 non-null  object        
 1   codmpio               334291 non-null  object        
 2   fecha_hechos_inicial  334346 non-null  datetime64[ns]
 3   fecha_hechos_final    334346 non-null  datetime64[ns]
 4   base_origen           334346 non-null  object        
 5   final                 334346 non-null  float64       
 6   inicial               334346 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 17.9+ MB


In [399]:
# Crea una Base de datos en memoria
con = sqlite3.connect(":memory:")

# Cruza los investigados con los intervalos
# Sube las tablas
KDE_result.to_sql("kde", con)
justicia.to_sql("justicia", con)

cursor = con.cursor()

cursor.execute("CREATE INDEX idx_kde_codmpio ON kde (codmpio)")
cursor.execute("CREATE INDEX idx_kde_inicial ON kde (inicial)")
cursor.execute("CREATE INDEX idx_kde_final ON kde (final)")
cursor.execute("CREATE INDEX idx_justicia_codmpio ON justicia (codmpio)")
cursor.execute("CREATE INDEX idx_justicia_inicial ON justicia (inicial)")
cursor.execute("CREATE INDEX idx_justicia_final ON justicia (final)")


# Execute the SQL query using pandas' read_sql_query function
query = """
SELECT
    kde.codmpio,
    kde.inicial,
    kde.final,
    COALESCE((
        SELECT 1 FROM justicia
        WHERE kde.codmpio = justicia.codmpio AND (
            (justicia.inicial <= kde.final)
            AND (justicia.final > kde.inicial)
        )
        LIMIT 1
    ),0) as investigado
FROM kde
"""
total = pd.read_sql_query(query, con)


In [402]:
# Convierte nuevamente los intervalos a fechas para facilitar su lectura
total.inicial = pd.to_datetime(total.inicial * 24 * 60 * 60 * 1000000000)
total.final = pd.to_datetime(total.final * 24 * 60 * 60 * 1000000000)

In [400]:
total["investigado"].value_counts()

0    309319
1    148335
Name: investigado, dtype: int64

In [401]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457654 entries, 0 to 457653
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   codmpio      457654 non-null  object
 1   inicial      457654 non-null  int64 
 2   final        457654 non-null  int64 
 3   investigado  457654 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 14.0+ MB


In [408]:
# Calcula el indicador
print(
    "resultado_" + kernel,
    round(total.investigado.mean()*100,2),total.investigado.sum()
)

resultado_gaussian 32.41 148335


In [404]:
# Guarda en un CSV el resultado
total.to_csv(p1 + "resultado_gaussian.csv", sep=";")

In [405]:
##RECICLAJE DE CÓDIGO
#convertir a fecha
# varp = ["fecha_hechos_final"]  
# for i in varp:
#     SIJYP_21[i] = SIJYP_21[i].astype(str).str.split(" ", n=1).str[0]

# mask = justicia[varp].apply(lambda x: any('2022-04-02 19:35:00' in str(i) for i in x), axis=0)
# mask
# datetime_column = "fecha_hechos_final"
# mask = justicia[datetime_column].apply(lambda x: '2022-04-02 19:35:00' in str(x))
# justicia.loc[mask]
#justicia["fecha_hechos_final"] = justicia["fecha_hechos_final"].str.replace('2022-04-02 19:35:00', '2022-04-02')